# Algoritmia

## Práctica Obligatoria 1

### Curso 2023 - 2024

###### Métodos Voraces

---

#### Autores:

- César Rodríguez Villagrá

---

Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**:

- Solamente puedes utilizar bibliotecas nativas (https://docs.python.org/es/3.8/library/index.html)
- Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.


In [19]:
#testeable
# Imports
import heapq

In [20]:
#testeable
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """

    def __init__(self, name, size):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size
        self.espectadores: dict = {}

    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """
        return hash((self.name, self.size))

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"Vídeo {self.name}, {self.size} GB"

    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"{self.name}-{self.size}GB"

    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """
        self.espectadores[country] = users

    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """
        return self.espectadores.get(country, 0)

In [21]:
#testeable
class ServidorCache:
    """
    Clase del servidor caché donde se almacenan parte de series/películas.
    """

    def __init__(self, identifier, country, capacity):
        """Instancia un Servidor de Caché

        Parameters
        ----------
        identifier : int
            Valor que identifica un servidor.
        country : str
            País donde está el servidor.
        capacity : int
            Cantidad de memoria de almacenamiento disponible.
        """
        self.identifier = identifier
        self.country = country
        self.capacity = capacity

        self.vidAlm: dict = {}
        self.cercano: ServidorCache = (None, float('inf'))

    def __hash__(self):
        """Genera el valor hash identificativo del servidor

        Returns
        -------
        int
            Valor hash
        """
        return hash((self.identifier, self.country, self.capacity))

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"Servidor {self.identifier} en {self.country} con capacidad de {self.capacity} GB"

    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"{self.identifier}-{self.country}-{self.capacity}GB"

    def rellena(self, videos):
        """Dada una colección de videos,
           seleccionar de cada uno cuanta cantidad (entre 0 y 1)
           se almacena en el servidor.
           Se ha de optimizar para que el tiempo de emisión
           sea el máximo posible.

        Parameters
        ----------
        videos : collection
            Colección de videos que se quieren almacenar en el servidor.
        """

        listaVideos = sorted(videos, key=lambda x: x.get_users(
            self.country), reverse=True)
        dataSize = 0
        for mejor in listaVideos:
            if dataSize + mejor.size <= self.capacity:
                dataSize += mejor.size
                self.vidAlm[mejor] = 1.0
            else:
                cantidad = (self.capacity-dataSize)/mejor.size
                self.vidAlm[mejor] = cantidad
                break
            

    def disponible(self, video):
        """Obtiene la cantidad de vídeo disponible en el servidor.

        Parameters
        ----------
        video : Video object
            Vídeo del cual se quiere saber la disponibilidad

        Returns
        -------
        float
            Cantidad del vídeo disponible          
        """
        return self.vidAlm.get(video, 0.0)

    def almacenados(self):
        """Material almacenado en el servidor

        Returns
        -------
        set
            Conjunto de tuplas (video, cantidad) de los videos ALMACENADOS en el servidor.          
        """
        return self.vidAlm.items()

    def tiempo_emision(self):
        """A partir de los datos almacenados
           devolver el tiempo de emisión
           siguiendo la fórmula: 
           \sum_{i}^{v} \text{espectadores}_i*\text{tamaño}_i*\text{porcionAlmacenada}_i

        Returns
        -------
        number
            Tiempo de emision disponible           
        """
        tEmision = 0
        for video in self.vidAlm.keys():
            tEmision += video.get_users(self.country) * \
                video.size*self.vidAlm[video]
        return tEmision

    def set_cercano(self, other: "ServidorCache", dist: int) -> None:
        """Actualiza el valor del servidor más cercano al actual, junto con la distancia al más cercano.

        Parameters
        ----------
        other : ServidorCache
            Servidor más cercano
        dist : int
            Distancia al servidor más cercano
        """
        self.cercano = (other, dist)

    def get_cercano(self) -> "ServidorCache":
        """Obtiene el servidor más cercano al actual.

        Returns
        -------
        ServidorCache
            Servidor más cercano
        """
        return self.cercano[0]

    def get_dist(self) -> int:
        """Distancia al servidor más cercano.

        Returns
        -------
        int
            La distancia
        """
        return self.cercano[1]

    def __lt__(self, other: str) -> bool:
        """Muestra si el servidor actual es menor que el pasado por parámetro
        (Suponiendo que el identifier sea único en cada servidor)
        El criterio para decidir que un servidor es menor que otro es arbitrario.

        Parameters
        ----------
        other : str
            El otro servidor a comparar

        Returns
        -------
        bool
            True si es menor, False en caso contrario
        """
        if isinstance(other, ServidorCache):
            return self.identifier < other.identifier
        return False

    def __eq__(self, other: str) -> bool:
        """Muestra si el servidor actual es igual que el pasado por parámetro.
        (Suponiendo que el identifier sea único en cada servidor)

        Parameters
        ----------
        other : str
            El otro servidor a comparar

        Returns
        -------
        bool
            True si es igual, False en caso contrario
        """
        if isinstance(other, ServidorCache):
            return self.identifier == other.identifier
        return False

In [22]:
#testeable
class ServidorMaestro:
    """
    Servidor central que gestiona las conexiones entre servidores cache
    """

    def __init__(self, servidores, distancias):
        """Instancia el servidor central

        Parameters
        ----------
        servidores : Iterable
            Conjunto de servidores cache disponibles
        distancias : dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """
        self.servidores = set(servidores)
        self.distancias: dict[ServidorCache,
                              dict[ServidorCache, int]] = distancias
        self.simplificado: dict[ServidorCache,
                                dict[ServidorCache, int]] = {}

    def get_grafo(self):
        """Devuelve el grafo de distancias recibido

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """

        return self.distancias

    def get_grafo_simplificado(self):
        """Devuelve el grafo de distancias simplificado

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """

        return self.simplificado

    def simplifica_grafo(self):
        """A partir del grafo de distancias
           hacer una simplificación de la estrucutra
           de datos para ahorrar espacio y tiempo.
        """
        noVisitados: set = self.servidores.copy()
        prioridad = []        

        while len(noVisitados) > 1:
            nodoAct = noVisitados.pop()
            for nodoDes in self.distancias[nodoAct]:
                if nodoDes in noVisitados:
                    heapq.heappush(
                        prioridad, (self.distancias[nodoAct][nodoDes], nodoAct, nodoDes))
            dist, nodoInicio, nodoFin = heapq.heappop(prioridad)
            if nodoInicio not in self.simplificado:
                self.simplificado[nodoInicio] = {}
            self.simplificado[nodoInicio][nodoFin] = dist



            
            for nodoDes in noVisitados:
                if self.distancias[nodoAct][nodoDes] < nodoDes.get_dist():
                    nodoDes.set_cercano(
                        nodoAct, self.distancias[nodoAct][nodoDes])
                if self.distancias[nodoDes][nodoAct] < nodoAct.get_dist():
                    nodoAct.set_cercano(
                        nodoDes, self.distancias[nodoDes][nodoAct])

    def mas_cercano(self, servidor):
        """Reporta el servidor más cercano al dado por parámetro

        Parameters
        ----------
        servidor : ServidorCache

        Returns
        -------
        ServidorCache
            Servidor más cercano
        """
        return servidor.get_cercano()

### Caso de ejemplo


In [23]:
import unittest
import json


def carga_dataset(data):
    with open(data) as f:
        test_datasets = json.load(f)

    videos = list()
    for v in test_datasets["videos"]:
        v_obj = Video(v["name"], v["size"])
        for c, u in v["users"].items():
            v_obj.set_users(c, u)
        videos.append(v_obj)

    servers = dict()
    for s in test_datasets["servers"]:
        servers[s["country"]] = ServidorCache(
            s["identifier"], s["country"], s["size"])

    pings = test_datasets["pings"]
    p_ = dict()
    for s in servers.values():
        p_[s] = dict()
        for p in pings[s.country]:
            p_[s][servers[p]] = pings[s.country][p]
    maestro = ServidorMaestro(servers.values(), p_)

    return videos, servers, maestro
    

In [24]:
class TestBasico(unittest.TestCase):

    def test_carga_simple(self):

        v, s, m = carga_dataset("toy.json")

        spain = s["Spain"]
        spain.rellena(v)
        self.assertEqual(spain.tiempo_emision(), 578000)
        almacenados = spain.almacenados()
        self.assertIn((v[3], 0.5), almacenados)

        m.simplifica_grafo()
        self.assertEqual(m.mas_cercano(s["Spain"]), s["France"])
        m.simplifica_grafo()
        self.assertEqual(m.mas_cercano(s["Spain"]), s["France"])
        self.assertEqual(m.mas_cercano(s["France"]), s["Spain"])
        print(m.get_grafo_simplificado())


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


{3-Ireland-1500GB: {1-Spain-1000GB: 150}, 4-Poland-2000GB: {2-France-2500GB: 250}, 2-France-2500GB: {1-Spain-1000GB: 50}}


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```

Los tests necesitan de las librerías `networkx` y `nbformat`

```bash
$ pip install networkx nbformat
```

###### Explicación de los tests

- `test_ejemplo`: Es el mismo que el caso de ejemplo.
- `test_ej1_emision_correcta`: Comprueba que el tiempo de emisión del servidor caché es correcto.
- `test_ej1_sin_espacio`: Comprueba que ante un servidor sin espacio, el tiempo de emisión es 0.
- `test_ej1_espacio_infinito`: Comprueba que ante un servidor con espacio infinito, el tiempo de emisión es el máximo.
- `test_ej1_pais_no_existe`: Comprueba que ante pais que no tiene servidor cache, el tiempo de emisión es 0.
- `test_ej2_estructura_datos_mas_simple`: Comprueba que la estructura de datos que se utiliza para almacenar la red de servidores es más simple que la original.
- `test_ej2_red_servidores_consistente`: Comprueba que la red de servidores es constitente con el mapa original, es decir, no hay conexiones nuevas y los costes son los mismos.
- `test_ej2_sistema_conexo`: Comprueba que la red de servidores cache es conexa.

---


### **Informe**

Contesta a las siguientes preguntas.


#### **Complejidad**

1. Método `ServidorCache.rellena`
   - **Complejidad temporal**: La complejidad de la función es O(n\*log(n)), donde n es el número de vídeos pasados por argumento, esta complejidad depende de la función de ordenar los vídeos pasados, que como Python utiliza Timsort, que en su peor caso tiene complejidad de O(n*log(n)), pero en su mejor caso O(n), en este caso la función tendría también O(n). Esto es debido a que tiene que recorrer todos los vídeos aunque la cantidad a almacenar sea 0, ya que se quiere indicar que para ese vídeo se almacena la cantidad de 0.
2. Método `ServidorMaestro.simplifica_grafo`
   - **Complejidad temporal**: La complejidad temporal es de O(c\*log(n)), donde c es el número de conexiones que hay (arcos), y n es el número de nodos.
  Esto se debe a que utilizo un montículo para almacenar las conexiones que tienen los nodos visitados, que se almacena cada conexión en un tiempo O(log(c)).
  Esto es en el caso en el que el número de conexiones sea menor que el número de nodos al cuadrado como es en el caso proporcionado en toy.json, en el otro caso la complejidad sería O(c\*log(a)).


#### **Servidores cache.**

- ¿La solución es óptima (maximiza siempre el tiempo de emisión) o es aproximada (encuentra un máximo local)?

Sí es óptima porque siempre llena el servidor con los vídeos que maximizen el tiempo de emisión, ya que se permite almacenar una porción de vídeo.

- ¿Qué ocurriría si solo se admitiese almacenar vídeos completos en cada servidor?

La solución no sería óptima ya que lo más probable es que quede algún hueco vacío sin usar, por lo que no garantizaría que se llene siempre, aunque queden vídeos por almacenar.


#### **Red de servidores cache**

- ¿La solución es óptima (la red es lo más simple posible) o es aproximada (encuentra un mínimo local)?

Es óptima, ya que la red de servidores obtenida tiene el mínimo valor de la suma de sus conexiones de todas las redes posibles, garantizando que la sulución sea la más corta posible.

- ¿Cómo afecta el número de conexiones entre servidores a la complejidad temporal del algoritmo empleado?

Si la cantidad de conexiones es mucho mayor que los nodos, la complejidad sería de O(c*log(c)) donde c es el número de conexiones, ya que el bucle en el que hace la insercción de las conexiones en el montículo se ejecutaría más veces.
En cuanto a la complejidad espacial, al haber más conexiones se necesita más espacio en el montículo para almacenarlas, para poder saber cual es la mínima de todas.